# Getting started

In [1]:
%load_ext autoreload
%autoreload 2

In [45]:
import dspy

from dspy.datasets import HotPotQA
from dspy.teleprompt import BootstrapFewShot
from dspy.evaluate.evaluate import Evaluate
from dsp.utils import deduplicate

ColBERTv2 server hosting a Wikipedia 2017 abstracts (first paragraph of each article)

In [3]:
colbertV2 = "http://20.102.90.50:2017/wiki17_abstracts"

In [4]:
turbo = dspy.OpenAI(model="gpt-3.5-turbo")

In [5]:
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url=colbertV2)
dspy.settings.configure(lm=turbo, rm=colbertv2_wiki17_abstracts)

## DSPy workflow

1. Collect a little bit of data
2. Write your program
3. Define some validation logic
4. Compile
5. Iterate

Get HotPotQA dataset

In [6]:
dataset = HotPotQA(
    train_seed=1, train_size=20, eval_seed=2023, dev_size=40, test_size=0
)

In [7]:
len(dataset.train), len(dataset.dev), len(dataset.test)

(20, 40, 0)

In [8]:
dataset.train[0].keys()

['question', 'answer']

In [9]:
dataset.dev[0].keys()

['question', 'answer', 'gold_titles']

### Questions and Answers

Show train example

In [10]:
train_example = dataset.train[0]
print(f"Question {train_example.question}")
print(f"Answer {train_example.answer}")

Question At My Window was released by which American singer-songwriter?
Answer John Townes Van Zandt


Show dev example

In [11]:
dev_example = dataset.dev[0]
print(f"Question {dev_example.question}")
print(f"Answer {dev_example.answer}")
print(f"Gold titles {dev_example.gold_titles}")

Question Are both Cangzhou and Qionghai in the Hebei province of China?
Answer no
Gold titles {'Cangzhou', 'Qionghai'}


### Inputs and Labels

Set inputs and labels

In [12]:
trainset = [x.with_inputs("question") for x in dataset.train]
devset = [x.with_inputs("question") for x in dataset.dev]

In [13]:
train_example = trainset[0]
dev_example = devset[0]

In [14]:
train_example.inputs().keys(), dev_example.inputs().keys()

(['question'], ['question'])

In [15]:
train_example.labels().keys(), dev_example.labels().keys()

(['answer'], ['answer', 'gold_titles'])

### Signatures

Create a signature

In [16]:
class BasicQA(dspy.Signature):
    "Answer questions with short factoid answers"

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

### Simple question-answer

Define the predictor

In [17]:
generate_answer = dspy.Predict(BasicQA)

In [18]:
pred = generate_answer(question=dev_example.question)
print(f"Question: {dev_example.question}")
print(f"Predicted Answer: {pred.answer}")

Question: Are both Cangzhou and Qionghai in the Hebei province of China?
Predicted Answer: No, Qionghai is in Hainan province.


In [19]:
turbo.inspect_history(n=1)




Answer questions with short factoid answers

---

Follow the following format.

Question: ${question}
Answer: often between 1 and 5 words

---

Question: Are both Cangzhou and Qionghai in the Hebei province of China?
Answer: No, Qionghai is in Hainan province.





'\n\n\nAnswer questions with short factoid answers\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nAnswer: often between 1 and 5 words\n\n---\n\nQuestion: Are both Cangzhou and Qionghai in the Hebei province of China?\nAnswer:\x1b No, Qionghai is in Hainan province.\x1b\n\n\n'

### Chain-of-thought question-answer

Define the predictor

In [20]:
generate_answer_cot = dspy.ChainOfThought(BasicQA)

In [21]:
pred = generate_answer_cot(question=dev_example.question)
print(f"Question: {dev_example.question}")
print(f"Thought: {pred.rationale}")
print(f"Predicted Answer: {pred.answer}")

Question: Are both Cangzhou and Qionghai in the Hebei province of China?
Thought: produce the answer. We can check the locations of Cangzhou and Qionghai.
Predicted Answer: No, Qionghai is in Hainan province.


### Retrieval

In [22]:
retrieve = dspy.Retrieve(k=3)
topK_passages = retrieve(dev_example.question).passages

In [23]:
print(f"Top {retrieve.k} passages for question: {dev_example.question}")

Top 3 passages for question: Are both Cangzhou and Qionghai in the Hebei province of China?


In [24]:
for idx, passage in enumerate(topK_passages):
    print(f"{idx + 1}: {passage}\n")

1: Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People's Republic of China. At the 2010 census, Cangzhou's built-up ("or metro") area made of Yunhe, Xinhua districts and Cang County largely being conurbated had a population of 1,205,814 inhabitants, while the prefecture-level administrative unit in total has a population of 7,134,062. It lies approximately 90 km from the major port city of Tianjin, and 180 km from Beijing.

2: Haixing County | Haixing County () is a county of southeastern Hebei province, China, bordering Shandong to the southeast. It is administered by Cangzhou City, and, , had a population of 220,000 residing in an area of 836 km2 . Both G18 Rongcheng–Wuhai Expressway and G25 Changchun–Shenzhen Expressway pass through the county.

3: Dongguang County | Dongguang County () is a county under the jurisdiction of Cangzhou City, in southeastern Hebei province, People's Republic of China, bordering Shandong to the southeast.



### Retrieval-augmented generation - RAG

In [25]:
class GenerateAnswer(dspy.Signature):
    "Answer questions with short factoid answers"

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

RAG dspy program

In [26]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

Teleprompter - optimizer that can take a program and bootstrap an effective prompt

In [27]:
def validate_context_answer(example, pred, trac=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM, answer_PM

In [28]:
teleprompter = BootstrapFewShot(metric=validate_context_answer)

In [29]:
compiled_rag = teleprompter.compile(RAG(), trainset=trainset)

 20%|████████████▏                                                | 4/20 [00:00<00:00, 267.38it/s]


Use compiled RAG program

In [31]:
my_question = "What castle did David Gregory inherit?"
pred = compiled_rag(my_question)

In [34]:
[c[:10] + "..." for c in pred.context]

['David Greg...', 'Gregory Ta...', 'David Greg...']

In [36]:
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved contexts: {[c[:200] + '...' for c in pred.context]}")

Question: What castle did David Gregory inherit?
Predicted Answer: Kinnairdy Castle
Retrieved contexts: ['David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinn...', 'Gregory Tarchaneiotes | Gregory Tarchaneiotes (Greek: Γρηγόριος Ταρχανειώτης , Italian: "Gregorio Tracanioto" or "Tracamoto" ) was a "protospatharius" and the long-reigning catepan of Italy from 998 t...', 'David Gregory (mathematician) | David Gregory (originally spelt Gregorie) FRS (? 1659 – 10 October 1708) was a Scottish mathematician and astronomer. He was professor of mathematics at the University ...']


In [38]:
# turbo.inspect_history(n=1)

View learned objects

In [39]:
for name, parameter in compiled_rag.named_predictors():
    print(name)
    print(parameter.demos[0])
    print()

generate_answer
Example({'augmented': True, 'context': ['At My Window (album) | At My Window is an album released by Folk/country singer-songwriter Townes Van Zandt in 1987. This was Van Zandt\'s first studio album in the nine years that followed 1978\'s "Flyin\' Shoes", and his only studio album recorded in the 1980s. Although the songwriter had become less prolific, this release showed that the quality of his material remained high.', 'Little Window | Little Window is the debut album of American singer-songwriter Baby Dee. The album was released in 2002 on the Durtro label. It was produced, composed, and performed entirely by Dee.', 'Windows and Walls | Windows and Walls is the eighth album by American singer-songwriter Dan Fogelberg, released in 1984 (see 1984 in music). The first single, "The Language of Love", reached 13 on the U.S. "Billboard" Hot 100 chart. Although the follow-up, "Believe in Me", missed the Top 40 of the pop chart, peaking at No. 48, it became the singer\'s fou

### Evaluate answers

In [41]:
# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_hotpotqa = Evaluate(
    devset=devset, num_threads=1, display_progress=True, display_table=5
)

# Evaluate the `compiled_rag` program with the `answer_exact_match` metric.
evaluate_on_hotpotqa(compiled_rag, metric=dspy.evaluate.answer_exact_match)

Average Metric: 23 / 40  (57.5): 100%|████████████████████████████| 40/40 [00:44<00:00,  1.10s/it]


,question,example_answer,gold_titles,context,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",No,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017 NHL Expansion Draft', '2017–18 Pittsburgh Penguins season'}",['2017–18 Pittsburgh Penguins season | The 2017–18 Pittsburgh Penguins season will be the 51st season for the National Hockey League ice hockey team that was...,National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","[""Æthelweard of East Anglia | Æthelweard (died 854) was a 9th-century king of East Anglia, the long-lived Anglo-Saxon kingdom which today includes the English counties...",Alfred the Great,False


57.5

Display additional columns

In [42]:
def gold_passages_retrieved(example, pred, trace=None):
    gold_titles = set(map(dspy.evaluate.normalize_text, example["gold_titles"]))
    found_titles = set(
        map(dspy.evaluate.normalize_text, [c.split(" | ")[0] for c in pred.context])
    )

    return gold_titles.issubset(found_titles)


compiled_rag_retrieval_score = evaluate_on_hotpotqa(
    compiled_rag, metric=gold_passages_retrieved
)

Average Metric: 11 / 40  (27.5): 100%|███████████████████████████| 40/40 [00:00<00:00, 303.45it/s]


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",No,False
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017 NHL Expansion Draft', '2017–18 Pittsburgh Penguins season'}",['2017–18 Pittsburgh Penguins season | The 2017–18 Pittsburgh Penguins season will be the 51st season for the National Hockey League ice hockey team that was...,National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","[""Æthelweard of East Anglia | Æthelweard (died 854) was a 9th-century king of East Anglia, the long-lived Anglo-Saxon kingdom which today includes the English counties...",Alfred the Great,False


### Multi-hop search

Create a signature

In [44]:
class GenerateSearchQuery(dspy.Signature):
    "Write a simple search query to answer a complex question"

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    query = dspy.OutputField()

Create program

In [47]:
class SimplifiedBaleen(dspy.Module):
    def __init__(self, passages_per_hop=3, max_hops=2):
        super().__init__()

        self.generate_query = [
            dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)
        ]
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.max_hops = max_hops

    def forward(self, question):
        context = []

        for hop in range(self.max_hops):
            query = self.generate_query[hop](context=context, question=question).query
            passages = self.retrieve(query).passages
            context = deduplicate(context + passages)

        pred = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=pred.answer)

Zero-shot learning

In [48]:
my_question = "How may storeys are in the castle that David Gregory inherited?"

uncompiled_baleen = SimplifiedBaleen()
pred = uncompiled_baleen(my_question)

In [49]:
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts: {[c[:200] + '...' for c in pred.context]}")

Question: How may storeys are in the castle that David Gregory inherited?
Predicted Answer: five
Retrieved Contexts: ['David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinn...', 'David Webster (architect) | David Webster (1885–1952) was a Scottish-Canadian architect best known for his designs of elementary schools in Saskatoon, Saskatchewan, Canada. His school designs were oft...', 'David Gregory (footballer, born 1970) | Born in Polstead, Gregory began his career at Ipswich Town, making 32 appearances between 1987–1995. He made two appearances on loan at Hereford United and thre...', 'Kinnairdy Castle | Kinnairdy Castle is a tower house, having five storeys and a garret, two miles south of Aberchirder, Aberdeenshire, Scotland. The alternative name is Old Kinnairdy....', 'Kinnordy House | Kinnordy House (alternative spellings: Kynnordy, Kina

In [52]:
def validate_context_and_answer_and_hops(example, pred, trace=None):
    if not dspy.evaluate.answer_exact_match(example, pred):
        return False
    if not dspy.evaluate.answer_passage_match(example, pred):
        return False

    hops = [example.question] + [
        outputs.query for *_, outputs in trace if "query" in outputs
    ]

    if max([len(h) for h in hops]) > 100:
        return False
    if any(
        dspy.evaluate.answer_exact_match_str(hops[idx], hops[:idx], frac=0.8)
        for idx in range(2, len(hops))
    ):
        return False

    return True

In [53]:
teleprompter = BootstrapFewShot(metric=validate_context_and_answer_and_hops)
compiled_baleen = teleprompter.compile(
    SimplifiedBaleen(), teacher=SimplifiedBaleen(passages_per_hop=2), trainset=trainset
)

100%|█████████████████████████████████████████████████████████████| 20/20 [01:11<00:00,  3.59s/it]


Compare uncompiled and compiled pipelines

In [54]:
uncompiled_baleen_retrieval_score = evaluate_on_hotpotqa(
    uncompiled_baleen, metric=gold_passages_retrieved
)

Average Metric: 22 / 40  (55.0): 100%|████████████████████████████| 40/40 [02:12<00:00,  3.32s/it]


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",No,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017 NHL Expansion Draft', '2017–18 Pittsburgh Penguins season'}","[""2017 NHL Expansion Draft | The 2017 NHL Expansion Draft was an expansion draft conducted by the National Hockey League on June 18–20, 2017 to...",National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,False
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","['River Esk, Dumfries and Galloway | The River Esk (Scottish Gaelic: ""Easg"" ), also called the Border Esk, is a river in Dumfries and Galloway,...",Cairn Water,False
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","['Æthelweard (son of Alfred) | Æthelweard (d. 920 or 922) was the younger son of King Alfred the Great and Ealhswith.', 'Æthelred Mucel | Æthelred...",King Alfred the Great,✔️ [True]


In [55]:
compiled_baleen_retrieval_score = evaluate_on_hotpotqa(
    compiled_baleen, metric=gold_passages_retrieved
)

Average Metric: 27 / 40  (67.5): 100%|████████████████████████████| 40/40 [02:11<00:00,  3.29s/it]


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",No,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017 NHL Expansion Draft', '2017–18 Pittsburgh Penguins season'}",['2017–18 Vegas Golden Knights season | The 2017–18 Vegas Golden Knights season will be the inaugural season for the Vegas Golden Knights. They will play...,National Hockey League,False
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,False
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}",['Prince Alfred of Great Britain | Prince Alfred (22 September 1780 – 20 August 1782) was a member of the British Royal Family as the...,King Alfred the Great,False


In [56]:
print(f"## Retrieval Score for RAG: {compiled_rag_retrieval_score}")
print(f"## Retrieval Score for uncompiled Baleen: {uncompiled_baleen_retrieval_score}")
print(f"## Retrieval Score for compiled Baleen: {compiled_baleen_retrieval_score}")

## Retrieval Score for RAG: 27.5
## Retrieval Score for uncompiled Baleen: 55.0
## Retrieval Score for compiled Baleen: 67.5
